## Off the shelf results with FLAN

In [1]:
%load_ext autoreload
%autoreload 2
from dataset.privacy_policies_dataset import DatasetLoader
from dataset.templates import PATTERNS

from src.privacyglue_tasks.policy_qa import load_policy_qa
from src.privacyglue_tasks.privacy_qa import load_privacy_qa
from src.utils.task_utils import policy_ie_file_mapping

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer, AutoModel
from custom_metrics import accuracy, compute_squad_metrics, exact_match_score, f1_score_compute
from datasets import load_from_disk, DatasetDict

## Privacy Policy Dataset Loading

In [2]:
dataset_loader = DatasetLoader()
dataset_loader.piextract_dataset

Found cached dataset text (/Users/loaynasser/.cache/huggingface/datasets/text/default-9bff2d11f4c1db3e/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset text (/Users/loaynasser/.cache/huggingface/datasets/text/default-feb562a0d55705b6/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/4209 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1041 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dataset_loader = DatasetLoader()                                                             │
│   2 dataset_loader.piextract_dataset                                                             │
│   3                                                                                              │
│                                                                                                  │
│ /Users/loaynasser/TUM/Praktikum/dataset/privacy_policies_dataset.py:27 in __init__               │
│                                                                                                  │
│   24 │   │   self.policy_qa_dataset = load_policy_qa('./dataset/privacyglue/policy_qa/')         │
│   25 │   │   self.privacy_qa_dataset = load_privacy_qa('./dataset/privacyglue/privacy_qa/')      │
│   26 │   │   self.policy_ie_a_dataset = load_policy_ie_a('./dataset/privacyglue/policy_ie_a/'    │
│ ❱ 27 │   │   self.policy_ie_b_dataset = load_policy_ie_b('./dataset/privacyglue/policy_ie_b/'    │
│   28 │   │   self.opp_115_dataset = load_opp_115('./dataset/privacyglue/opp_115/')               │
│   29 │   │   self.piextract_dataset = load_piextract('./dataset/privacyglue/piextract/')         │
│   30 │   │   self.policy_detection_dataset = load_policy_detection('./dataset/privacyglue/pol    │
│                                                                                                  │
│ /Users/loaynasser/TUM/Praktikum/src/privacyglue_tasks/policy_ie_b.py:46 in load_policy_ie_b      │
│                                                                                                  │
│    43 │   data_files = policy_ie_file_mapping(directory, "seq.in")                               │
│    44 │                                                                                          │
│    45 │   # load tokens which are common for all sub-tasks                                       │
│ ❱  46 │   tokens = datasets.load_dataset(                                                        │
│    47 │   │   "text", data_files=policy_ie_file_mapping(directory, "seq.in")                     │
│    48 │   ).map(lambda example: {"tokens": example["text"].split()}, remove_columns=["text"])    │
│    49                                                                                            │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/datasets/load.py:1 │
│ 773 in load_dataset                                                                              │
│                                                                                                  │
│   1770 │   )                                                                                     │
│   1771 │                                                                                         │
│   1772 │   # Create a dataset builder                                                            │
│ ❱ 1773 │   builder_instance = load_dataset_builder(                                              │
│   1774 │   │   path=path,                                                                        │
│   1775 │   │   name=name,                                                                        │
│   1776 │   │   data_dir=data_dir,                                                                │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/datasets/load.py:1 │
│ 502 in load_dataset_builder                                

In [172]:
dataset_loader.piextract_dataset["train"]["COLLECT"][500]["tags"]
dataset_loader.piextract_dataset["train"]["COLLECT"][500]["text"]

'Given the following text, identify the relevant information to be collected and mark it with the appropriate tags:\n\n Text: This information may be used to offer you optimized TED.com experiences based on your device or location .'

In [174]:
# Create a new DatasetDict with only the type-I datasets
type_i_dataset_dict = DatasetDict({
    'train': dataset_loader.policy_ie_b_dataset['train']['type-II'],
    'validation': dataset_loader.policy_ie_b_dataset['validation']['type-II'],
    'test': dataset_loader.policy_ie_b_dataset['test']['type-II'],
})

type_i_dataset_dict.save_to_disk("policy_ie_b_dataset_type_II")

Saving the dataset (0/1 shards):   0%|          | 0/4209 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1041 [00:00<?, ? examples/s]

In [159]:
# Create a new DatasetDict with only the type-I datasets
type_ii_dataset_dict = DatasetDict({
    'train': dataset_loader.policy_ie_b_dataset['train']['type-II'],
    'validation': dataset_loader.policy_ie_b_dataset['validation']['type-II'],
    'test': dataset_loader.policy_ie_b_dataset['test']['type-II'],
})

type_ii_dataset_dict.save_to_disk("policy_ie_b_dataset_type_II")

Saving the dataset (0/1 shards):   0%|          | 0/4209 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1041 [00:00<?, ? examples/s]

In [176]:
# Create a new DatasetDict with only the type-I datasets
piextract_dataset_dict = DatasetDict({
    'train': dataset_loader.piextract_dataset['train']['NOT_COLLECT'],
    'validation': dataset_loader.piextract_dataset['validation']['NOT_COLLECT'],
    'test': dataset_loader.piextract_dataset['test']['NOT_COLLECT'],
})

piextract_dataset_dict.save_to_disk("piextract_dataset_not_collect")

Saving the dataset (0/1 shards):   0%|          | 0/2579 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/456 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1029 [00:00<?, ? examples/s]

In [178]:
# Create a new DatasetDict with only the type-I datasets
piextract_dataset_dict = DatasetDict({
    'train': dataset_loader.piextract_dataset['train']['NOT_SHARE'],
    'validation': dataset_loader.piextract_dataset['validation']['NOT_SHARE'],
    'test': dataset_loader.piextract_dataset['test']['NOT_SHARE'],
})

piextract_dataset_dict.save_to_disk("piextract_dataset_not_share")

Saving the dataset (0/1 shards):   0%|          | 0/2579 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/456 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1029 [00:00<?, ? examples/s]

In [ ]:
dataset_loader.privacy_qa_dataset.save_to_disk("privacy_qa_dataset")

In [ ]:
dataset_loader.piextract_dataset["train"]["COLLECT"][200]

In [ ]:
dataset_loader.piextract_dataset["train"]["SHARE"][200]

In [20]:
dataset_loader.policy_detection_dataset.save_to_disk("policy_detection_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/773 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/137 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/391 [00:00<?, ? examples/s]

In [73]:
dataset_loader.opp_115_dataset.save_to_disk("opp_115_dataset2")

Saving the dataset (0/1 shards):   0%|          | 0/2185 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/550 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/697 [00:00<?, ? examples/s]

In [48]:
dataset_loader.policy_ie_a_dataset.save_to_disk("policy_ie_a_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/4209 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1041 [00:00<?, ? examples/s]

## Initializing the model & tokenizer

In [10]:
#Small Model & Tokenizer Loading
small_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')
small_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-small')


In [5]:
#Base Model & Tokenizer Loading
base_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
base_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-base')

In [ ]:
#Large Model & Tokenizer Loading
large_flan_model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-large')
large_flan_tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large')

: 

In [ ]:
large_flan_model.config

In [ ]:
#XLarge Model & Tokenizer Loading
xlarge_flan_model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xl')
xlarge_flan_tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-xl')

In [ ]:
#Loading finetuned model
model = AutoModel.from_pretrained('./normal_combined_115_qa_ie_model')

In [ ]:
# prompt for OPP-115 with T5

t5_prepared_text =  f'multilabel classification: \n\n Text:\n{dataset_loader.opp_115_dataset["train"][1]["text"] } \n\nOPTIONS:\n-Data Retention \n-Data Security \n-Do Not Track \n-First Party Collection/Use \n-International and Specific Audiences \n-Introductory/Generic \n-Policy Change \n-Practice not covered \n-Privacy contact information \n-Third Party Sharing/Collection \n-User Access, Edit and Deletion \n-User Choice/Control'
print(t5_prepared_text,dataset_loader.opp_115_dataset["train"][1]["label"] )
input_ids = large_flan_tokenizer.encode(dataset_loader.opp_115_dataset["train"][1]["text"]  , return_tensors="np")

# q&a 
opp_115_output_ids = large_flan_model.generate(
    input_ids
)

output = large_flan_tokenizer.decode(opp_115_output_ids[0], skip_special_tokens=True)

print (f"Response: \n{output}")

## OPP-115 --- Zero-Shot Learning
A total of 115 privacy policies were selected based on their corresponding company's popularity on Google Trends. 
Given an input paragraph segment of a privacy policy. the goal is to predict one or more data practive categories.

In [49]:
dataset_loader.opp_115_dataset['train'][2]['text']

"Classify the following text into a list of relevant categories.\n\n Text:\n *Web Beacons: Military Web pages and the Web pages of our partners also utilize electronic images known as Web beacons (sometimes called single-pixel gifs, clear gifs or action tags) that allow us to collect certain information about users' visits to these Web pages (including the number of users who have visited those pages), measure and improve the effectiveness of advertisements and job postings, and deliver co-branded services. Web beacons are not used to access your personally identifiable information; they are a technique used to compile aggregated statistics about web site usage. Web beacons collect only a limited set of information including a cookie number, time and date of page view and a description of the page on which the Web beacon resides. This information may be shared with third party processors for processing on behalf of Military, as well as with clients, partners, and other third parties so

In [ ]:
# prompt for OPP-115 with T5

t5_prepared_text =  dataset_loader.opp_115_dataset['train'][3]['text']
input_ids = large_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
input_ids = large_flan_tokenizer(dataset_loader.opp_115_dataset['train'][1]['label'][0], max_length=128, truncation=True, padding="max_length", return_tensors="pt")
# q&a 
opp_115_output_ids = large_flan_model.generate(
    input_ids
)

output = large_flan_tokenizer.decode(opp_115_output_ids[0], skip_special_tokens=True)

print (f"Response: \n{output}")

In [ ]:
import torch 
device = "cuda:0" if torch.cuda.is_available() else "cpu"

from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from peft import PeftModel, PeftConfig


peft_model_id = "./FLAN_OPP115_results"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = model.to(device)
model.eval()
# 'complaint'


In [98]:
dataset_loader.policy_ie_a_dataset['train'][1]

{'text': 'Text:\nThis is done so we could re-consider Applicants for other positions and opportunities at DeviantArt ; so we could use their Applicants Information as reference for future applications submitted by them ; and in case the Applicant is hired , for additional employment and business purposes related to his / her work at DeviantArt .\nWhich topic is this text about?\nOPTIONS:\n-Other\n-Data Collection Usage\n-Data Security Protection\n-Data Sharing Disclosure\n-Data Storage Retention Deletion',
 'label': 'Data Storage Retention Deletion'}

In [ ]:
dataset_loader.piextract_dataset["train"]["SHARE"][100]

In [ ]:
inputs = tokenizer(' '.join(dataset_loader.piextract_dataset["train"]["SHARE"][100]), return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

In [137]:
PATTERNS['piextract'][1]

'Given the following text, identify the relevant information to be shared and mark it with the appropriate tags:\n\n Text:For complete guidelines on children\'s information collection , please visit the PBS KIDS PRIVACY POLICY. \n Tags: ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]\n\n Text: We do not release any of your contact information to third parties unless we have your permission . \n Tags: ["O", "O", "O", "O", "O", "O", "B-SHARE", "I-SHARE", "I-SHARE", "O", "O", "O", "O", "O", "O", "O", "O", "O"]\n\n Text: {text} \n Tags:'

In [172]:
prompt = "Given the following text, identify the relevant information to be shared and mark it with the appropriate tags:\n\n"

example1_text = "For complete guidelines on children's information collection, please visit the PBS KIDS PRIVACY POLICY."
example1_tags = '["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]'

example2_text = "We do not release any of your contact information to third parties unless we have your permission."
example2_tags = '["O", "O", "O", "O", "O", "O", "B-SHARE", "I-SHARE", "I-SHARE", "O", "O", "O", "O", "O", "O", "O", "O", "O"]'

out = 'Output Tags Meaning: \n "O": "The label O denotes that the word is not part of a named entity" \n "B-SHARE": "Share Entity Beginning" \n "I-SHARE": "Share Entity Inside"'
formatted_prompt = prompt + "\nText: " + example1_text + "\nTags: " + example1_tags + "\n\nText: " + example2_text + "\nTags: " + example2_tags + "\n\nText: {text}\nTags:\n "+out
formatted_prompt

'Given the following text, identify the relevant information to be shared and mark it with the appropriate tags:\n\n\nText: For complete guidelines on children\'s information collection, please visit the PBS KIDS PRIVACY POLICY.\nTags: ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]\n\nText: We do not release any of your contact information to third parties unless we have your permission.\nTags: ["O", "O", "O", "O", "O", "O", "B-SHARE", "I-SHARE", "I-SHARE", "O", "O", "O", "O", "O", "O", "O", "O", "O"]\n\nText: {text}\nTags:\n Output Tags Meaning: \n "O": "The label O denotes that the word is not part of a named entity" \n "B-SHARE": "Share Entity Beginning" \n "I-SHARE": "Share Entity Inside"'

In [173]:
formatted_prompt.format(text="hi")

'Given the following text, identify the relevant information to be shared and mark it with the appropriate tags:\n\n\nText: For complete guidelines on children\'s information collection, please visit the PBS KIDS PRIVACY POLICY.\nTags: ["O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O", "O"]\n\nText: We do not release any of your contact information to third parties unless we have your permission.\nTags: ["O", "O", "O", "O", "O", "O", "B-SHARE", "I-SHARE", "I-SHARE", "O", "O", "O", "O", "O", "O", "O", "O", "O"]\n\nText: hi\nTags:\n Output Tags Meaning: \n "O": "The label O denotes that the word is not part of a named entity" \n "B-SHARE": "Share Entity Beginning" \n "I-SHARE": "Share Entity Inside"'

In [ ]:
dataset_loader.policy_qa_dataset 

In [46]:
import statistics
f1_scores = []

In [47]:
# Define evaluation metrics

# Prepare evaluation data
dataset = dataset_loader.opp_115_dataset["train"] # or any other appropriate evaluation subset

# Evaluate zero shot predictions
total_examples = len(dataset)

small_correct_predictions = 0
base_correct_predictions = 0
large_correct_predictions = 0
xlarge_correct_predictions = 0
xxlarge_correct_predictions = 0

predictions = []
large_predictions = []
i = 1
for example in dataset:
    t5_prepared_text = example["text"]
    print(t5_prepared_text)
    # small_input_ids = small_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    # opp_115_small_output_ids = small_flan_model.generate(small_input_ids)
    # small_output = small_flan_tokenizer.decode(opp_115_small_output_ids[0], skip_special_tokens=True)
    # small_predicted_label = small_output
    
    # predictions.append(small_predicted_label)
    
    # base_input_ids = base_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    # opp_115_base_output_ids = base_flan_model.generate(base_input_ids)
    # base_output = base_flan_tokenizer.decode(opp_115_base_output_ids[0], skip_special_tokens=True)
    # base_predicted_label = base_output
    
    
    large_input_ids = large_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    opp_115_large_output_ids = large_flan_model.generate(large_input_ids)
    large_output = large_flan_tokenizer.decode(opp_115_large_output_ids[0], skip_special_tokens=True)
    large_predicted_label = large_output
    
    # large_predictions.append(large_predicted_label)
    
    # xlarge_input_ids = xlarge_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    # opp_115_xlarge_output_ids = xlarge_flan_model.generate(xlarge_input_ids)
    # xlarge_output = xlarge_flan_tokenizer.decode(opp_115_xlarge_output_ids[0], skip_special_tokens=True)
    # xlarge_predicted_label = xlarge_output
    
    # xlarge_predictions.append(xlarge_predicted_label)
    
    ground_truth_label = example['answer']  # Modify this line based on your specific use case

    print("predicted",large_predicted_label)
    print("ground truth",ground_truth_label)
    print("--------------------------------")
    
    if large_predicted_label in ground_truth_label:
        large_correct_predictions += 1
    
    accur = accuracy(large_correct_predictions,i)
    f1 = f1_score_compute(large_predicted_label, ground_truth_label)
    f1_scores.append(f1)
    print("accuracy", accur)
    print("f1",statistics.mean(f1_scores))
    i = i + 1
        
    # if base_predicted_label in ground_truth_label:
    #     base_correct_predictions += 1
    
    # if large_predicted_label in ground_truth_label:
    #     large_correct_predictions += 1
        
    # if xlarge_predicted_label in ground_truth_label:
    #     xlarge_correct_predictions += 1    

# You can also calculate and print other metrics based on your requirements
# ...


Answer this question: 

How do they collect information about users? 

context: INFORMATION WE COLLECT ABOUT YOU When you interact with SIDEARM Services, we collect: (1) personal information you supply when you subscribe, order, complete a form or survey, register for one of our Web Sites, enter a contest or provide your email address and (2) non-personal information transmitted through technology, including tracking information, which is also collected by third parties. 

answer:


/Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


predicted (1) personal information you supply when you subscribe, order, complete a form or survey, register
ground truth personal information you supply when you subscribe, order, complete a form or survey, register for one of our Web Sites, enter a contest or provide your email address and
--------------------------------
accuracy 0.0
f1 0.5909090909090909
Answer this question: 

What type of information about me does the website collect? 

context: INFORMATION WE COLLECT ABOUT YOU When you interact with SIDEARM Services, we collect: (1) personal information you supply when you subscribe, order, complete a form or survey, register for one of our Web Sites, enter a contest or provide your email address and (2) non-personal information transmitted through technology, including tracking information, which is also collected by third parties. 

answer:
predicted personal information
ground truth personal information
--------------------------------
accuracy 0.5
f1 0.7954545454545454
Answe

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:35                                                                                   │
│                                                                                                  │
│   32 │                                                                                           │
│   33 │                                                                                           │
│   34 │   large_input_ids = large_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")    │
│ ❱ 35 │   opp_115_large_output_ids = large_flan_model.generate(large_input_ids)                   │
│   36 │   large_output = large_flan_tokenizer.decode(opp_115_large_output_ids[0], skip_special    │
│   37 │   large_predicted_label = large_output                                                    │
│   38                                                                                             │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/torch/utils/_conte │
│ xtlib.py:115 in decorate_context                                                                 │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/transformers/gener │
│ ation/utils.py:1329 in generate                                                                  │
│                                                                                                  │
│   1326 │   │   if self.config.is_encoder_decoder and "encoder_outputs" not in model_kwargs:      │
│   1327 │   │   │   # if model is encoder decoder encoder_outputs are created                     │
│   1328 │   │   │   # and added to `model_kwargs`                                                 │
│ ❱ 1329 │   │   │   model_kwargs = self._prepare_encoder_decoder_kwargs_for_generation(           │
│   1330 │   │   │   │   inputs_tensor, model_kwargs, model_input_name                             │
│   1331 │   │   │   )                                                                             │
│   1332                                                                                           │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/transformers/gener │
│ ation/utils.py:642 in _prepare_encoder_decoder_kwargs_for_generation                             │
│                                                                                                  │
│    639 │   │   model_input_name = model_input_name if model_input_name is not None else self.ma  │
│    640 │   │   encoder_kwargs["return_dict"] = True                                              │
│    641 │   │   encoder_kwargs[model_input_name] = inputs_tensor                                  │
│ ❱  642 │   │   model_kwargs["encoder_outputs"]: ModelOutput

In [65]:
# Define evaluation metrics

# Prepare evaluation data
dataset = dataset_loader.opp_115_dataset['validation']  # or any other appropriate evaluation subset

# Evaluate zero shot predictions
total_examples = len(dataset)

small_correct_predictions = 0
base_correct_predictions = 0
large_correct_predictions = 0
xlarge_correct_predictions = 0
xxlarge_correct_predictions = 0

predictions = []
large_predictions = []
i = 1
for example in dataset:
    t5_prepared_text = example['text']
    
    # small_input_ids = small_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    # opp_115_small_output_ids = small_flan_model.generate(small_input_ids)
    # small_output = small_flan_tokenizer.decode(opp_115_small_output_ids[0], skip_special_tokens=True)
    # small_predicted_label = small_output
    
    # predictions.append(small_predicted_label)
    
    # base_input_ids = base_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    # opp_115_base_output_ids = base_flan_model.generate(base_input_ids)
    # base_output = base_flan_tokenizer.decode(opp_115_base_output_ids[0], skip_special_tokens=True)
    # base_predicted_label = base_output
    
    
    large_input_ids = large_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    opp_115_large_output_ids = large_flan_model.generate(large_input_ids)
    large_output = large_flan_tokenizer.decode(opp_115_large_output_ids[0], skip_special_tokens=True)
    large_predicted_label = large_output
    
    # large_predictions.append(large_predicted_label)
    
    # xlarge_input_ids = xlarge_flan_tokenizer.encode(t5_prepared_text, return_tensors="pt")
    # opp_115_xlarge_output_ids = xlarge_flan_model.generate(xlarge_input_ids)
    # xlarge_output = xlarge_flan_tokenizer.decode(opp_115_xlarge_output_ids[0], skip_special_tokens=True)
    # xlarge_predicted_label = xlarge_output
    
    # xlarge_predictions.append(xlarge_predicted_label)
    
    ground_truth_label = example['label']  # Modify this line based on your specific use case

    print("predicted",large_predicted_label)
    print("ground truth",ground_truth_label)
    print("--------------------------------")
    
    if large_predicted_label in ground_truth_label:
        large_correct_predictions += 1
    
    accur = accuracy(large_correct_predictions,i)
    print("accuracy", accur)
    i = i + 1
        
    # if base_predicted_label in ground_truth_label:
    #     base_correct_predictions += 1
    
    # if large_predicted_label in ground_truth_label:
    #     large_correct_predictions += 1
        
    # if xlarge_predicted_label in ground_truth_label:
    #     xlarge_correct_predictions += 1    

# You can also calculate and print other metrics based on your requirements
# ...


predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 1.0
predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 1.0
predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 1.0
predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 1.0
predicted First Party Collection/Use
ground truth Third Party Sharing/Collection
--------------------------------
accuracy 0.8
predicted Data Security
ground truth User Access, Edit and Deletion
--------------------------------
accuracy 0.6666666666666666
predicted Third Party Sharing/Collection
ground truth Third Party Sharing/Collection
--------------------------------
accuracy 0.7142857142857143
predicted Data Security
ground truth Data Security
--------------------------------
accuracy 0.75
predicte

Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors


predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 0.5483870967741935
predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 0.5625
predicted Third Party Sharing/Collection
ground truth Privacy contact information
--------------------------------
accuracy 0.5454545454545454
predicted Third Party Sharing/Collection
ground truth Third Party Sharing/Collection
--------------------------------
accuracy 0.5588235294117647
predicted First Party Collection/Use
ground truth First Party Collection/Use
--------------------------------
accuracy 0.5714285714285714
predicted First Party Collection/Use
ground truth Third Party Sharing/Collection
--------------------------------
accuracy 0.5555555555555556
predicted Policy Change
ground truth Policy Change
--------------------------------
accuracy 0.5675675675675675
predicted First Party Collection/Use
ground truth Inter

In [ ]:
# Calculate evaluation metrics
small_accuracy = small_correct_predictions / total_examples
base_accuracy = base_correct_predictions / total_examples
large_accuracy = large_correct_predictions / total_examples
xlarge_accuracy = xlarge_correct_predictions / total_examples

# Print the evaluation results
print(f"Small Accuracy: {small_accuracy}")
print(f"Base Accuracy: {base_accuracy}")
print(f"Large Accuracy: {large_accuracy}")§
print(f"XLarge Accuracy: {xlarge_accuracy}")

Accuracy: 0.45720823798627


## Policy Q/A - Question/Answering

In [ ]:
input_ids = large_flan_tokenizer.encode(
    'question: Where does Sinan live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = large_flan_model.generate(
    input_ids,
    early_stopping=True
)

output = large_flan_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

In [ ]:
input_ids = base_tokenizer.encode(
    'question: Where does Matt live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
Boston


In [ ]:
# Sanity check with random prompts

input_ids = base_tokenizer.encode(
    'prompt1: Where does Matt live? prompt2: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
prompt2


## Fine-tuning T5 for abstractive summarization

In [10]:
from transformers import pipeline, T5ForConditionalGeneration, TrainingArguments, Trainer, \
                         DataCollatorForSeq2Seq
import pandas as pd
from datasets import Dataset
import random
import peft

In [45]:
import torch

torch.has_mps

device = torch.device('cpu')
model.to(device)
device

device(type='cpu')

In [11]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
import pandas as pd

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(large_flan_model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817


trainable params: 4718592 || all params: 787868672 || trainable%: 0.5989059049678777


In [12]:
# https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv

reviews = pd.read_csv('Reviews.csv')

# Pre-processing step
# Punctuation is important in grammar and important for complex decoding architectures to know when to stop!
def add_punc(s):
    if s[-1] not in ('.', '!', '?'):
        s = s + '.'
    return s

reviews.dropna(inplace=True)

reviews['Summary'] = reviews['Summary'].map(add_punc)

print(reviews.shape)

reviews.head()

(568401, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food.,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised.,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all.",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine.,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy.,Great taffy at a great price. There was a wid...


In [13]:
reviews = reviews[(reviews['Summary'].str.len() < 100) & (reviews['Summary'].str.len() >= 30)]

reviews.shape

(157784, 10)

In [14]:
import random
random.seed(0)

reviews_dataset = Dataset.from_pandas(reviews.astype(str).sample(5000))
reviews_dataset


Dataset({
    features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__'],
    num_rows: 5000
})

In [15]:
# We have a prompt but only as a prefix in the encoder
prefix = "summarize: "

# we will manually add our own labels because unlike GPT, we cannot assume the labels are based on the inputs
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = large_flan_tokenizer(inputs, max_length=1024, truncation=True)

    labels = large_flan_tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
tokenized_reviews_dataset = reviews_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [17]:
tokenized_reviews_dataset = tokenized_reviews_dataset.train_test_split(test_size=.1)
tokenized_reviews_dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

In [19]:
# Data collator specifically for generic sequence to sequence tasks
# Use when we are translating one sequence to another like translation, summarization, etc
data_collator = DataCollatorForSeq2Seq(tokenizer=large_flan_tokenizer, model=model)

In [54]:
!pip install transformers[torch]

zsh:1: no matches found: transformers[torch]


In [55]:
training_args = TrainingArguments(
    output_dir="./t5_summary_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    logging_steps=50,
    save_strategy='epoch'
)

In [20]:
training_args = TrainingArguments(
    output_dir="./t5_summary_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    load_best_model_at_end=True,
    logging_steps=50,
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_reviews_dataset["train"],
    eval_dataset=tokenized_reviews_dataset["test"],
    data_collator=data_collator,
)

trainer.evaluate()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:20                                                                                   │
│                                                                                                  │
│   17 │   data_collator=data_collator,                                                            │
│   18 )                                                                                           │
│   19                                                                                             │
│ ❱ 20 trainer.evaluate()                                                                          │
│   21                                                                                             │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/transformers/train │
│ er.py:3053 in evaluate                                                                           │
│                                                                                                  │
│   3050 │   │   start_time = time.time()                                                          │
│   3051 │   │                                                                                     │
│   3052 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3053 │   │   output = eval_loop(                                                               │
│   3054 │   │   │   eval_dataloader,                                                              │
│   3055 │   │   │   description="Evaluation",                                                     │
│   3056 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/transformers/train │
│ er.py:3245 in evaluation_loop                                                                    │
│                                                                                                  │
│   3242 │   │   │   │   │   batch_size = observed_batch_size                                      │
│   3243 │   │   │                                                                                 │
│   3244 │   │   │   # Prediction step                                                             │
│ ❱ 3245 │   │   │   loss, logits, labels = self.prediction_step(model, inputs, prediction_loss_o  │
│   3246 │   │   │   inputs_decode = self._prepare_input(inputs["input_ids"]) if args.include_inp  │
│   3247 │   │   │                                                                                 │
│   3248 │   │   │   if is_torch_tpu_available():                                                  │
│                                                                                                  │
│ /Users/loaynasser/miniconda3/envs/new_tensorflow/lib/python3.10/site-packages/transformers/train │
│ er.py:3503 in prediction_step                                                                    │
│                                                                                                  │
│   3500 │   │   │   else:                                                                         │
│   3501 │   │   │   │   if has_labels or loss_without_labels:                                     │
│   3502 │   │   │   │   │   with self.compute_loss_context_manager():                             │
│ ❱ 3503 │   │   │   │   │   │   loss, outputs = self.compute_loss(model, inputs, return_outputs=  │
│   3504 │   │   │   │   │   loss = loss.mean().detach()                                           │
│   3505 │   │   │   │   │                                   

In [ ]:
trainer.train()

/Users/loaynasser/miniconda3/envs/tensorflow/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.592700,3.335303


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Id, ProfileName, Summary, Score, Time, __index_level_0__, UserId, ProductId, Text, HelpfulnessDenominator, HelpfulnessNumerator. If Id, ProfileName, Summary, Score, Time, __index_level_0__, UserId, ProductId, Text, HelpfulnessDenominator, HelpfulnessNumerator are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 3.3603246212005615,
 'eval_runtime': 33.7016,
 'eval_samples_per_second': 14.836,
 'eval_steps_per_second': 0.475,
 'epoch': 2.0}

In [ ]:
trainer.save_model()

Saving model checkpoint to ./t5_summary_results
Configuration saved in ./t5_summary_results/config.json
Model weights saved in ./t5_summary_results/pytorch_model.bin


In [ ]:
loaded_model = T5ForConditionalGeneration.from_pretrained('./t5_summary_results')

# summarization pipeline prepends a default prefix of summarize: 
generator = pipeline(
    'summarization', model=loaded_model, tokenizer=base_tokenizer
)

loading configuration file ./t5_summary_results/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_sto

In [ ]:
sam = reviews.sample(1)

print(sam['Summary'])

text = sam['Text'].tolist()[0]
text

418718    best beans for the buck you will find.
Name: Summary, dtype: object


'Very happy with this coffee. mellow, and rich, organic and fair trade, what more can you ask for? Oh yeah, what a great deal! THanks'

In [ ]:
# Generate a summary
generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'Great coffee.'}]

In [ ]:
# Try the base t5 on the same text
base_generator = pipeline(
    'summarization', model='t5-small', tokenizer='t5-small'
)

loading configuration file config.json from cache at /Users/sinanozdemir/.cache/huggingface/hub/models--t5-small/snapshots/9507060efcd5189100109e25df8326eb07274a36/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram

In [ ]:
# Summary is a bit more extractive than our fine-tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'THanks coffee is a great coffee . mel'}]